## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [27]:
DATA_URL = "http://az.lib.ru/n/nekrasow_n_a/text_1840_pevitza.shtml"

Устанавливаем библиотеки

In [2]:
import warnings
warnings.filterwarnings('ignore')
!pip -q install rnnmorph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 KB 862.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Создаём объект морфологического анализатора `RNNMorph`

In [3]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [28]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [29]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Некрасов Николай Алексеевич. Певица</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/n/nekrasow_n_a/>Некрасов Николай Алексеевич</a><br>\r\nПевица</h2>\r\n\r'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [30]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [31]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Некрасов Николай Алексеевич. Певица\n\n\n\nНекрасов Николай Алексеевич\r\nПевица\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n[Пом'

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [38]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)
print(len(set(word_tokenize(cleaned_text))))

4633


[nltk_data] Downloading package punkt to /Users/nikitav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [35]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent] 
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться



NameError: name 'predictor' is not defined

Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [ ]:
len(predictions)

577

In [33]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens) 


NameError: name 'predictions' is not defined

In [34]:
nltk.FreqDist(non_uniq_tokens)

NameError: name 'non_uniq_tokens' is not defined

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам. 

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75. 

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [22]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikitav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что']

In [13]:
%pip install spacy
%python3 -m spacy download ru_core_news_sm

Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%python3` not found (But cell magic `%%python3` exists, did you mean that instead?).


In [20]:
import spacy
from spacy import displacy
nlp = spacy.load("ru_core_news_sm")
nlp.pipe_names

doc = nlp(cleaned_text)
for token in doc:
    print(token.text)




Lib.ru/Классика
:
Толстой
Алексей
Константинович
.
Семья
вурдалака





Толстой
Алексей
Константинович


Семья
вурдалака




Lib.ru/Классика
:





[
Регистрация
]

 




[
Найти
]


[
Рейтинги
]


[
Обсуждения
]


[
Новинки
]


[
Обзоры
]


[
Помощь
]











Комментарии
:
12
,
последний
от
31/05/2020
.


Толстой
Алексей
Константинович

 
(
yes@lib.ru
)


Год
:
1839


Обновлено
:
22/11/2018
.
57k
.
Статистика
.


Глава
:
Проза
,
Фантастика
Проза




Скачать
FB2









Оценка
:
6.86
*
79



 
Ваша
оценка
:


шедевр


замечательно


очень
хорошо


хорошо


нормально


Не
читал


терпимо


посредственно


плохо


очень
плохо


не
читать
























Алексей
Константинович
Толстой
.
Семья
вурдалака



---------------------------------------------------------------

    
Оригинал
здесь
:
Библиотека
OCR
Альдебаран


---------------------------------------------------------------

      
               
Неизданный
отрывок
из
записок
неизвестного

     
	         
Примечани

Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)
